In [64]:
import pandas as pd
import os

cvs_fp = r'C:\Users\iya\Dev\CVM\CVS\2022'
cvs_tour_fn = r'sandag_cvs_tour.xlsx'
tnc_tour_fn = r'sandag_tnc_tour.xlsx'
cvs_df_fn = r'cv_20221102_sandag_draftfinal.xlsx'
db_fn = r'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'

pd.options.display.float_format = '{:20,.2f}'.format

establishment_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Establishment Data')
trip_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Trip Data')
tour_data = pd.read_excel(os.path.join(cvs_fp, cvs_tour_fn), sheet_name='sandag_cvs_tour')
veh_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Vehicle Data')

In [54]:
tour_data['com_veh_dri_tri'] = tour_data[
    ['company_id', 'vehicle_id', 'driver_id', 'trip_number']
    ].astype(str).agg('-'.join, axis=1)

trip_data['com_veh_dri_tri'] = trip_data[
    ['company_id', 'vehicle_id', 'driver_id', 'trip_number']
    ].astype(str).agg('-'.join, axis=1)

In [55]:
def custom_lambda(x):
    vals = tour_data[tour_data.com_veh_dri_tri == x]['TRAVEL_DISTANCE'].values
    if len(vals) == 1: return vals[0]
    elif len(vals) == 2: 
        if vals[1] != vals[1]:
            if vals[0] == vals[0]: return vals[0]
            else: print(f'trips with conflicting distances: {x}')
        else: print(f'trips with conflicting distances: {x}')
    else: print(f'trips with more than two assigned distances: {x}')

trip_data['trip_dis'] = trip_data.com_veh_dri_tri.apply(
    lambda x: custom_lambda(x)
    )

trips with conflicting distances: 173100-133-188-0
trips with conflicting distances: 173100-133-188-0
trips with conflicting distances: 173100-133-188-1
trips with conflicting distances: 173100-133-188-1
trips with conflicting distances: 173100-133-188-2
trips with conflicting distances: 173100-133-188-2


In [41]:
a = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)+\
    '_'+ tour_data.STOP_SEQ.astype(str)
a.nunique()

12770

In [56]:
trip_data = trip_data.merge(
    veh_data[['vehicle_classification', 'id']],
    left_on='vehicle_id',
    right_on='id',
    validate='many_to_one',
    how='left'
    )
trip_data = trip_data[trip_data.vehicle_classification != 4]

In [57]:
trip_data_ = trip_data[trip_data.arrival_time.notna()]


In [58]:
trip = trip_data_.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')
trip = trip[trip.vehicle_classification != 4]
gc = trip[
    ['base_location_Industry Group', 'Most Likely Estimate Weight Factor']
    ].groupby('base_location_Industry Group').sum()
gc

,Most Likely Estimate Weight Factor
base_location_Industry Group,
1,"9,828.80"
2,"106,340.04"
3,"17,881.25"
4,"81,389.15"
5,"69,012.34"
6,"168,813.25"
7,"88,476.31"
8,"216,093.05"
9,"483,683.83"


In [59]:
gc.reset_index()['Most Likely Estimate Weight Factor'].sum()

1643728.4444367785

In [7]:
len(trip_data_) #matches

10687

In [11]:
max(tour_id) #matches

2213

In [8]:
tour_id = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

started, ended, prev = False, False, (None, None)
for se in list(zip(start, end)):
    if se == (True, False): 
        started, ended = True, False
        counter += 1
    elif se == (False, True):
        started, ended = False, True
    # if ended and not started: counter += 1
    tour_id.append(counter)

In [9]:
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

In [10]:
tour_data['tour_id'] = tour_id

light, med, hvy = list(range(1, 4)), list(range(5, 8)), [4, 8]
def classing(num):
    if num in light: return 'light'
    elif num in med: return 'med'
    else: return 'hvy'
tour_data['VehClass_name'] = tour_data.VehClass.apply(lambda x: classing(x))

In [ ]:
trip_data[trip_data.]

In [44]:
tour_data_ = tour_data[tour_data.VehClass_name == 'hvy']

tour_data_[['tour_id','Most.Likely.Estimate.Weight.Factor']].groupby('tour_id').max().reset_index()['Most.Likely.Estimate.Weight.Factor'].sum()

48878.16900914829

In [50]:
tour_data_[['tour_id','Most.Likely.Estimate.Weight.Factor']].drop_duplicates()['Most.Likely.Estimate.Weight.Factor'].sum()

48878.16900914829

In [11]:
employees = establishment_data[['base_location_Industry Group', 'no_of_emp_work', 'employees_fulltime_count']]
employees = employees['employees_fulltime_count'].sum()
employees

46272

In [12]:
2213/46272

0.04782589903181189

In [15]:
len(trip_data_)/2213

4.829191143244465

In [16]:
10687/2213

4.829191143244465

In [17]:
trip_data_

,company_id,vehicle_id,driver_id,trip_number,trip_load_status,trip_load_weight,activity_type,activity_type_other,placetype,placetype_other,...,travel_date,arrival_time,departure_time,used_other_vehicle,participation_type,vehicle_id.1,Lower Estimate Weight Factor,Most Likely Estimate Weight Factor,Upper Estimate Weight Factor,com_veh_dri_tri
1,100012,2062,2589,1,NaN,NaN,11,NaN,6,NaN,...,2022-09-30,10:20:00,11:10:00,2,online,2062,109.69,177.83,198.18,100012-2062-2589-1
2,100012,2062,2589,2,NaN,NaN,1,NaN,11,NaN,...,2022-09-30,11:22:00,11:40:00,2,online,2062,109.69,177.83,198.18,100012-2062-2589-2
3,100012,2062,2589,3,NaN,NaN,10,NaN,96,DOG PARK,...,2022-09-30,12:00:00,14:00:00,2,online,2062,109.69,177.83,198.18,100012-2062-2589-3
4,100012,2062,2589,4,NaN,NaN,1,NaN,11,NaN,...,2022-09-30,14:20:00,NaN,2,online,2062,109.69,177.83,198.18,100012-2062-2589-4
6,100012,2063,2590,1,NaN,NaN,10,NaN,96,DOG PARK,...,2022-09-30,14:11:00,16:45:00,2,online,2063,109.69,177.83,198.18,100012-2063-2590-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12256,600154,1859,2398,2,NaN,NaN,6,NaN,20,NaN,...,2022-10-11,08:46:00,09:11:00,2,smartphone,1859,178.85,247.70,254.34,600154-1859-2398-2
12257,600154,1859,2398,3,NaN,NaN,5,NaN,20,NaN,...,2022-10-11,09:44:00,10:05:00,2,smartphone,1859,178.85,247.70,254.34,600154-1859-2398-3
12258,600154,1859,2398,4,NaN,NaN,10,NaN,96,ROAD SIDE,...,2022-10-11,10:38:00,10:54:00,2,smartphone,1859,178.85,247.70,254.34,600154-1859-2398-4
12259,600154,1859,2398,5,NaN,NaN,5,NaN,20,NaN,...,2022-10-11,11:27:00,11:46:00,2,smartphone,1859,178.85,247.70,254.34,600154-1859-2398-5


In [65]:
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

vmt = tour_data[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group', 'TRAVEL_DISTANCE']
].dropna(how='any')
vmt['weighted_distance'] = vmt['Most.Likely.Estimate.Weight.Factor'] * vmt.TRAVEL_DISTANCE
vmt_ = vmt[['weighted_distance', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum().reset_index()
vmt_

,base_location_Industry Group,weighted_distance
0,1,"94,294.62"
1,2,"1,455,627.51"
2,3,"94,476.61"
3,4,"662,266.36"
4,5,"1,054,351.58"
5,6,"1,996,970.71"
6,7,"753,000.36"
7,8,"2,000,252.90"
8,9,"2,830,538.79"
9,10,"3,226,709.41"


In [66]:
vmt_.weighted_distance.sum()

14422966.8588167

In [ ]:
# #manual fixing tour_data tour_id
# tour_data.loc[(tour_data.vehicle_id == 117)&
#               (tour_data.TOUR_NUM == 3)&
#               (tour_data.STOP_SEQ == 2), 'tour_id'] = 775
# tour_data.loc[(tour_data.vehicle_id == 176)&
#               (tour_data.TOUR_NUM == 2)&
#               (tour_data.STOP_SEQ == 2)&
#               (tour_data.trip_number == 4),  'tour_id'] = 1530

# a = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
# a = {v: k+1 for k, v in enumerate(a.unique())}
# tour_data_ = tour_data[['company_id', 'driver_id', 'vehicle_id','TOUR_NUM', 'trip_number', 'STOP_SEQ', 'TOUR_START', 'TOUR_END']].copy()
# tour_data_['tour_id2'] = tour_id
# tour_data_['tour_id3'] = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
# tour_data_['tour_id3_'] = tour_data_.tour_id3.apply(lambda x: a[x])
# tour_data_.to_csv('tour_id_verification.csv')

In [142]:
tour_data.sort_values(['vehicle_id', 'TOUR_NUM','STOP_SEQ'])

,company_id,driver_id,trip_number,vehicle_id,STOP_BASE_NOTBASE,TOUR_NUM,STOP_SEQ,trip_load_status,trip_load_weight,activity_type,...,ARRIVE_TIME,TOUR_START,TOUR_STOPS,TOUR_END,STOP_NOT_RETURN,First_Sched_Stop,base_location_Industry Group,tour_id1,tour_id2,tour_id3
4777,120941,58,0,20,1,1,1,4.00,NaN,1,...,NaN,True,2,False,False,NaN,6,877,885,20_1
4778,120941,58,1,20,2,1,2,NaN,NaN,96,...,NaN,False,2,False,True,NaN,6,877,885,20_1
4779,120941,58,2,20,1,1,3,NaN,NaN,1,...,NaN,False,2,True,False,NaN,6,877,885,20_1
4780,120941,59,0,21,1,1,1,4.00,NaN,1,...,NaN,True,10,False,False,3.00,6,878,886,21_1
4781,120941,59,1,21,2,1,2,NaN,NaN,2,...,NaN,False,10,False,True,3.00,6,878,886,21_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388,117393,354,1,2129,2,1,2,NaN,NaN,8,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
3389,117393,354,2,2129,2,1,3,NaN,NaN,11,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
3390,117393,354,3,2129,2,1,4,NaN,NaN,8,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
3391,117393,354,4,2129,2,1,5,NaN,NaN,2,...,NaN,False,5,False,True,2.00,7,633,640,2129_1


In [143]:
tour_id_ = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

started, ended, prev = False, False, (None, None)
for se in list(zip(start, end)):
    if se == (True, False): 
        started, ended = True, False
        counter += 1
    elif se == (False, True):
        started, ended = False, True
    #if ended and not started: counter += 1
    tour_id_.append(counter)

In [144]:
tour_id = []
counter = 0
c, d, tp, v, tr = tour_data.company_id, tour_data.driver_id, tour_data.trip_number, tour_data.vehicle_id, tour_data.TOUR_NUM
c_, d_, tp_, v_, tr_ = None, None, None, None, None
for i in range(len(c)):
    if tr[i] != tr_: counter += 1
    else:
        if tp[i] == 0: counter += 1
    c_, d_, tp_, v_, tr_ = c[i], d[i], tp[i], v[i], tr[i]
    tour_id.append(counter)

In [145]:
a = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
a = {v: k+1 for k, v in enumerate(a.unique())}


In [139]:
tour_data_ = tour_data[['company_id', 'driver_id', 'vehicle_id','TOUR_NUM', 'trip_number', 'STOP_SEQ', 'TOUR_START', 'TOUR_END']].copy()
tour_data_['tour_id1'] = tour_id_
tour_data_['tour_id2'] = tour_id
tour_data_['tour_id3'] = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
tour_data_['tour_id3_']= tour_data_.tour_id3.apply(lambda x: a[x])
tour_data_

,company_id,driver_id,vehicle_id,TOUR_NUM,trip_number,STOP_SEQ,TOUR_START,TOUR_END,tour_id1,tour_id2,tour_id3,tour_id3_
0,100012,2589,2062,1,0,1,True,False,1,1,2062_1,1
1,100012,2589,2062,1,1,2,False,False,1,1,2062_1,1
2,100012,2589,2062,1,2,3,False,True,1,1,2062_1,1
3,100012,2589,2062,2,2,1,True,False,2,2,2062_2,2
4,100012,2589,2062,2,3,2,False,False,2,2,2062_2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
12895,600154,2398,1859,1,2,3,False,False,2213,2216,1859_1,2180
12896,600154,2398,1859,1,3,4,False,False,2213,2216,1859_1,2180
12897,600154,2398,1859,1,4,5,False,False,2213,2216,1859_1,2180
12898,600154,2398,1859,1,5,6,False,False,2213,2216,1859_1,2180


In [140]:
tour_data_.to_csv('tour_id_verification.csv')

In [147]:
max(tour_id_)

2213

In [151]:
max(a.values())

2180

In [152]:
tour_data['tour_id3'] = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
tour_data['tour_id1'] = tour_id_
tour_data['tour_id2'] = tour_id

In [154]:
tour_1 = tour_data[['Most.Likely.Estimate.Weight.Factor', 'tour_id1']].groupby('tour_id1').mean().reset_index()
tour_1 = tour_1.merge(
    tour_data[['base_location_Industry Group', 'tour_id1']].groupby('tour_id1').mean(), 
    on='tour_id1',
    how='left',
    validate='one_to_many'
    )
tour_1[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group']
].groupby('base_location_Industry Group').sum().reset_index()['Most.Likely.Estimate.Weight.Factor'].sum()

321209.2729924468

In [119]:
tour_data.used_other_vehicle.value_counts()

used_other_vehicle
2    12894
1        6
Name: count, dtype: int64

In [141]:
tour_data.sort_values(['vehicle_id', 'TOUR_NUM','STOP_SEQ'])

,company_id,driver_id,trip_number,vehicle_id,STOP_BASE_NOTBASE,TOUR_NUM,STOP_SEQ,trip_load_status,trip_load_weight,activity_type,...,ARRIVE_TIME,TOUR_START,TOUR_STOPS,TOUR_END,STOP_NOT_RETURN,First_Sched_Stop,base_location_Industry Group,tour_id1,tour_id2,tour_id3
4777,120941,58,0,20,1,1,1,4.00,NaN,1,...,NaN,True,2,False,False,NaN,6,877,885,20_1
4778,120941,58,1,20,2,1,2,NaN,NaN,96,...,NaN,False,2,False,True,NaN,6,877,885,20_1
4779,120941,58,2,20,1,1,3,NaN,NaN,1,...,NaN,False,2,True,False,NaN,6,877,885,20_1
4780,120941,59,0,21,1,1,1,4.00,NaN,1,...,NaN,True,10,False,False,3.00,6,878,886,21_1
4781,120941,59,1,21,2,1,2,NaN,NaN,2,...,NaN,False,10,False,True,3.00,6,878,886,21_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388,117393,354,1,2129,2,1,2,NaN,NaN,8,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
3389,117393,354,2,2129,2,1,3,NaN,NaN,11,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
3390,117393,354,3,2129,2,1,4,NaN,NaN,8,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
3391,117393,354,4,2129,2,1,5,NaN,NaN,2,...,NaN,False,5,False,True,2.00,7,633,640,2129_1
